In [13]:
pip install selenium 


Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup as bs
import urllib.request
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

In [11]:
chrome_driver_path = 'chromedriver.exe'
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path=chrome_driver_path, options = options)

In [12]:
driver.get('https://haraj.com.sa/en/tags/%D8%AD%D8%B1%D8%A7%D8%AC%20%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B1%D8%A7%D8%AA')
button = driver.find_element(By.XPATH, '//button[@data-testid="posts-load-more"]')
button.click()
time.sleep(10)   
html_content = driver.page_source
bsoup = bs(html_content, 'html.parser')

In [13]:
cars_posts = bsoup.find_all( class_ = 'box-border items-center overflow-hidden text-ellipsis')


In [14]:
hrefs = []
for element in cars_posts:
    a = element.find_all('a')
    if len(hrefs) > 40:
        break
    for href in a:
        hrefs.append('https://haraj.com.sa'+href.get('href'))

In [15]:
posts = []

for i in range(len(hrefs)):
    src = urllib.request.urlopen(hrefs[i]).read()
    bsoup = bs(src, 'html.parser')
    posts.append((bsoup.find_all('h1')[0].text, 
                 bsoup.find_all('span', class_ = 'city')[0].text, 
                 bsoup.find_all('a', class_ = 'flex min-h-[24px] max-w-[14ch] items-center overflow-hidden text-text-primary sm:max-w-full')[0].text,
                 bsoup.find_all('article')[0].text.replace('\n', ' ')))   
posts



[('Distinctive painting (RR 2002)',
  'Madinah',
  'basim aleisaa',
  'Parts & Accessories Special panel  لوحة مميزة قفلين ر ص ر 2002 على السوم :   ( رقم الجوال يظهر في الخانة المخصصة )  '),
 ('kia k8',
  'Riyadh',
  'ابو صالح 4595',
  'Kia NS 2022 Gasoline mileage: 34 KM  كيا k8 استاندر الممشى 34الف الموتر وكاله وتحت الضمان  ضمان الجبر 5 سنين الكيلو مفتوح  جميع الصيانه في الوكاله والموتر شرط ما يشكي من اي شي.   التواصل واتس '),
 ('beltz filter',
  'Hail',
  'عبـدالله 41902',
  'Parts & Accessories Spare parts  فلتر بلتز استعمال قليل وصوته قوي وطرب '),
 ('plate for sale',
  'Buqayq',
  'sahib ahmed',
  'Parts & Accessories Spare parts  &rlm;على السوم &rlm;لوحة مميز '),
 ('Private plate for sale, refurbished, unexamined, special number, S.K. 20',
  'Khaybar',
  'saleh_amr',
  'Toyota Land Cruiser 2020 Automatic Gasoline mileage: 300 KM 4WD  البيع الوحه خصوصي ط ص ك 20             السعر :00 '),
 ('Upholstery details',
  'Riyadh',
  '88عمرumar',
  'Parts & Accessories Spare parts  التفصيل 

In [17]:
headers = ['Item', 'Location', 'Seller', 'Description']
df = pd.DataFrame(posts, columns = headers)
df.to_csv('output.csv', index=False)
df.head()

,Item,Location,Seller,Description
0,Distinctive painting (RR 2002),Madinah,basim aleisaa,Parts & Accessories Special panel لوحة مميزة ...
1,kia k8,Riyadh,ابو صالح 4595,Kia NS 2022 Gasoline mileage: 34 KM كيا k8 اس...
2,beltz filter,Hail,عبـدالله 41902,Parts & Accessories Spare parts فلتر بلتز است...
3,plate for sale,Buqayq,sahib ahmed,Parts & Accessories Spare parts &rlm;على السو...
4,"Private plate for sale, refurbished, unexamine...",Khaybar,saleh_amr,Toyota Land Cruiser 2020 Automatic Gasoline mi...
